In [26]:
# Importamos librerías de análisis de datos
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
# pd.set_option('mode.chained_assignment', None) # Deshabilita SettingWithCopyWarning. Ojo.

In [109]:
# Cargamos el dataframe
df = pd.read_csv('data/train.csv', index_col='id', parse_dates=['fecha'])

In [110]:
df.shape

(240000, 22)

In [111]:
df.head(20)

,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
id,,,,,,,,,,,,,,,,,,,,,,
254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,nan,2.00,1.00,2.00,80.00,80.00,23533.00,nan,nan,2015-08-23,0.00,0.00,0.00,0.00,0.00,2273000.00
53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.00,3.00,2.00,2.00,268.00,180.00,24514.00,19.31,-99.23,2013-06-28,0.00,0.00,0.00,1.00,1.00,3600000.00
247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.00,3.00,2.00,2.00,144.00,166.00,48551.00,nan,nan,2015-10-17,0.00,0.00,0.00,0.00,0.00,1200000.00
209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.00,2.00,1.00,1.00,63.00,67.00,53666.00,19.30,-99.69,2012-03-09,0.00,0.00,0.00,1.00,1.00,650000.00
185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.00,2.00,1.00,1.00,95.00,95.00,47835.00,nan,nan,2016-06-07,0.00,0.00,0.00,0.00,0.00,1150000.00
126147,departamento en venta taxqueña,"amplio departamento, estancia de sala y comedo...",Apartamento,Condominio Tlalpan 2B,Coyoacán,Distrito Federal,5.00,2.00,1.00,1.00,75.00,90.00,23650.00,19.30,-99.15,2014-03-18,0.00,0.00,0.00,0.00,1.00,1100000.00
139233,de oportunidad casa en san lorenzo,"ubicada en esquina, pertenece san lorenzo agen...",Casa,NaN,Oaxaca de Juárez,Oaxaca,nan,3.00,1.00,2.00,140.00,160.00,73510.00,17.14,-96.80,2016-02-23,0.00,0.00,0.00,0.00,0.00,1150000.00
5013,casa emilia en venta en selvamar playa del carmen,casa emilia en venta playa del carmenfracciona...,Casa,condominio el trebol,Playa del Carmen,Quintana Roo,2.00,4.00,2.00,3.00,293.00,293.00,130510.00,20.67,-87.04,2016-10-20,0.00,0.00,0.00,0.00,0.00,4200000.00
44962,pre- venta preciosos depas 2 recamaras con sub...,<p>pre-venta de preciosos departamento ecologi...,Apartamento,BUENAVISTA DEPTOS CON SUBSIDIO,Villa de Alvarez,Colima,1.00,2.00,1.00,1.00,58.00,nan,9010.00,nan,nan,2014-01-06,0.00,0.00,0.00,1.00,1.00,310000.00


In [112]:
# Veo la cantidad de elementos nulos de cada columna
display(df.isnull().sum())

titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

# Optimizando los tipos de datos
## Integers
<span style="font-weight:600;">Lleno los elementos nulos de tipo tipo float con el valor -1, para que sea un valor valido para la conversion a int, pero que sea facil de identificar como valor vacio, al no coinicidr con los valores posibles de las columnas seleccionadas</span>

In [113]:
df[['antiguedad','habitaciones','garages','banos','idzona']] = df[['antiguedad','habitaciones','garages','banos','idzona']].fillna(-1)

<span style="font-weight:600;">Cambio el tipo de tadas las columnas que son float y que en verdad se representan en int</span>

In [114]:
df[['antiguedad', 'habitaciones', 'garages', 'banos', 'idzona', 'precio']] = pd.DataFrame(df, columns=['antiguedad', 'habitaciones', 'garages', 'banos', 'idzona', 'precio'], dtype=int)

<span style="font-weight:600;">Paso del tippo de dato int64 que se coloco en el paso anterior , y lo downcasteo a el minimo posible para cada columna (int32, int16, etc)</span>

In [115]:
df[['antiguedad', 'habitaciones', 'garages', 'banos', 'idzona', 'precio']] = df[['antiguedad', 'habitaciones', 'garages', 'banos', 'idzona', 'precio']].apply(pd.to_numeric, downcast='integer')

## Floats
<span style="font-weight:600;">Lleno los elementos nulos de tipo tipo float con el valor -1, para que sea un valor valido, pero que sea facil de identificar como valor vacio, al no coinicidr con los valores posibles de las columnas seleccionadas</span>

In [116]:
df[['metroscubiertos','metrostotales']] = df[['metroscubiertos','metrostotales']].fillna(-1)

<span style="font-weight:600;">Downcasteo a el minimo posible para cada columna (float32, float16, etc)</span>

In [117]:
df[['metroscubiertos', 'metrostotales']] = df[['metroscubiertos', 'metrostotales']].apply(pd.to_numeric,downcast='float')

## Booleans y Category
<span style="font-weight:600;">Cambio el tipo de dato de tadas las columnas que en verdad se representan en bool y la columna que se representa como caregory</span>

In [118]:
df[['gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']] = pd.DataFrame(df, columns=['gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos'], dtype=bool)
df = df.astype({'tipodepropiedad': 'category'})

## Resultado
### Tipo de datos final para cada columna

In [119]:
df.dtypes

titulo                                object
descripcion                           object
tipodepropiedad                     category
direccion                             object
ciudad                                object
provincia                             object
antiguedad                              int8
habitaciones                            int8
garages                                 int8
banos                                   int8
metroscubiertos                      float32
metrostotales                        float32
idzona                                 int32
lat                                  float64
lng                                  float64
fecha                         datetime64[ns]
gimnasio                                bool
usosmultiples                           bool
piscina                                 bool
escuelascercanas                        bool
centroscomercialescercanos              bool
precio                                 int32
dtype: obj

### Cantidad de nulos por columna

In [120]:
display(df.isnull().sum())

titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                         0
habitaciones                       0
garages                            0
banos                              0
metroscubiertos                    0
metrostotales                      0
idzona                             0
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

# Analisis de datos

In [121]:
# Retiro las columnas de "lat" y "lng" dado que en el 50% de los datos estan vacios
# df = df.drop(columns=['lat', 'lng'])

### Ordeno las filas por Fecha , id y precio

In [122]:
df = df.sort_values(by=["fecha","id","precio"])

### Conteo de la cantidad de publicaciones de venta por año

In [135]:
anios = pd.DataFrame(df[['fecha']], columns=['fecha'])
anios = anios["fecha"].groupby(anios['fecha'].dt.year).agg({'count'})
anios

,count
fecha,
2012,23534
2013,30386
2014,40572
2015,51470
2016,94038


### Conteo de la cantidad de publicaciones de venta por provincias

In [136]:
df.head()

,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
id,,,,,,,,,,,,,,,,,,,,,,
814,casa en venta en bosques de la herradura,bonita casa en tres niveles construida para pe...,Casa,2Âª CERRADA DE BOSQUE DE JIQUILPAN,Huixquilucan,Edo. de México,23,4,2,-1,-1.00,400.00,55570,nan,nan,2012-01-01,False,False,False,True,False,4500000
4369,casa en venta en guadalajara,excelente casa en muy buena ubicacion............,Casa,Isla Española 2603,Guadalajara,Jalisco,40,7,1,2,119.00,119.00,-1,20.64,-103.38,2012-01-01,False,False,False,True,True,1900000
7128,casa en condominio en venta en fraccionamiento...,"magnifica casa en condominio privado, con coch...",Casa en condominio,Carr Antiguo Aeropuerto,Querétaro,Querétaro,4,3,-1,2,430.00,-1.00,84007,nan,nan,2012-01-01,False,False,False,True,False,5500000
9702,casa en condominio en venta en san andrés cholula,hermosa y vanguardista casa por forjadores jun...,Casa en condominio,Melchor Ocampo 12,San Andrés Cholula,Puebla,0,3,2,2,102.00,84.00,-1,nan,nan,2012-01-01,False,False,False,False,False,900000
23940,departamento en venta en interlomas,"hermoso departamento seminuevo, moderno y muy...",Apartamento,Jesus del Monte,Huixquilucan,Edo. de México,5,3,-1,3,301.00,-1.00,55584,nan,nan,2012-01-01,False,False,True,True,False,5700000


In [140]:
provincias = pd.DataFrame(df, columns=['provincia'])
provincias = provincias["provincia"].groupby(provincias['provincia']).agg({'count'})
provincias = provincias.sort_values(by=["count"])
provincias.head()

,count
provincia,
Zacatecas,94
Campeche,263
Oaxaca,711
Tlaxcala,839
Tabasco,994
